# CelebA Dataset

### File Characteristics

img_align_celeba (sized images)
 - 202,599 images

identity_CelebA
 - ALL lines: imge_id.jpg (ex. 000001.jpg) person_id

list_attr_celeba
 - 1st line: number of images
 - 2nd line: Categories
 - 3rd+ lines: imge_id.jpg (ex. 000001.jpg) 1 or -1 for each category

list_landmarks_align_celeba
 - 1st line: number of images
 - 2nd line: categories (ie. lefteye_x righteye_x etc)
 - 3rd_ lines: img_id.jpg (ex. 000001.jpg) number for position

#### File Links
Image Dataset: "C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\img_align_celeba"

Identity Annotations: "C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\CELEBA_annotations\identity_CelebA.txt"

Attribute Annoatations: "C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\CELEBA_annotations\list_attr_celeba.txt"

Landmark Location Annotations: "C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\CELEBA_annotations\list_landmarks_align_celeba.txt"




In [1]:
import tensorflow as tf
from keras import layers
import numpy as np
import cv2
import os
import csv

In [2]:
# Example opening one image in greyscale
image = cv2.imread(r"C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\img_align_celeba\000001.jpg", 0)

# Display first image
cv2.namedWindow('First Image', cv2.WINDOW_NORMAL)   # Create window
cv2.imshow('image', image)                          # Display image
cv2.waitKey(0)                                      # Destroy window when press 0
cv2.destroyAllWindows()

# The 2D Matrix of the image values. 
npimage = np.array(image)
np.set_printoptions()
print("First Image, Grey Scale")
print(np.shape(npimage))
print(npimage)

First Image, Grey Scale
(218, 178)
[[233 233 233 ... 232 241 241]
 [233 233 233 ... 234 241 241]
 [233 233 233 ... 236 242 242]
 ...
 [ 88  63  93 ...  72  73  73]
 [ 77  85 113 ...  66  68  68]
 [115 151 192 ...  66  68  68]]


# Dividing up the dataset (Priminary)
Total image count: 202,599

Preliminary Training:   000001.jpg - 010000.jpg Count: 10,000 <br>

Image dimensions 178 x 218

First goal: Hat Detection

In [3]:
# Hat detection
# Wearing Hat is at 35 index in second row
# Wearing Hat is at 36 index (with first value being the ID of the image)
# Total of 40 Attributes, Example Row:
# ['000001.jpg' '-1' '1' '1' '-1' '-1' '-1' '-1' '-1' '-1' '-1' '-1' '1' '-1' '-1' '-1' '-1' '-1' '-1' '1' '1' '-1' '1' '-1' '-1' '1' '-1' '-1' '1' '-1' '-1' '-1' '1' '1' '-1' '1' '-1' '1' '-1' '-1' '1']
# CSV File format
# *     Label 1     Label 2     ...     Label n
# imgID value 1     value 2     ...     value n
# imgID value 1     value 2     ...     value n
labels_csv_path = r"C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\CELEBA_annotations\list_attr_celeba_CSV.csv"
image_dataset_directory = r"C:\Users\johns\OneDrive\Desktop\ML\MLData\CELEBA_dataset\img_align_celeba"

# open csv and read the file names and labels
with open(labels_csv_path, 'r') as file:
    reader = csv.reader(file)
    next(reader) # skip first header row

    # Create lists to store file names and labels
    file_names = []
    labels = []

    for i, line in enumerate(reader):
        if i>= 20000:                   # The first 10000 values
            break

        file_name = line[0]             # 0 is where the file name is
        label = line[36]                # 36 is where the Wearing_Hat label is

        file_names.append(file_name)    # ADDING FILE NAME TO LIST
        labels.append(label)            # ADDING LABEL TO LIST  

# Load the images and the labels
images = []
for file_name in file_names:
    image_path = f"{image_dataset_directory}\{file_name}"
    img = cv2.imread(image_path, 0) # Grayscale read, shouldn't need color for hat detection
    images.append(img)

images = np.array(images)
images = images / 255.0 # Normalize to 0 to 1
labels_pre = np.array(labels)
labels = [1 if label == '1' else -1 for label in labels_pre] # NO string format
labels = np.array(labels)


In [5]:
# images --> a 10000 length list where each item is a array with gray scale values normalized 0 to 1
# labels --> a 10000 length list where each index is 1 or -1 corresponding to images
image_shape = np.shape(images) # (10000, 218, 178)
label_shape = np.shape(labels) # (10000,)
input_shape = (image_shape[1], image_shape[2], 1)


In [19]:
# Create CNN Model Architecutre

model = tf.keras.Sequential ([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape = (218, 178, 1)),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2,2,)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_16 (Conv2D)          (None, 216, 176, 32)      320       
                                                                 
 max_pooling2d_16 (MaxPoolin  (None, 108, 88, 32)      0         
 g2D)                                                            
                                                                 
 conv2d_17 (Conv2D)          (None, 106, 86, 64)       18496     
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 53, 43, 64)       0         
 g2D)                                                            
                                                                 
 conv2d_18 (Conv2D)          (None, 51, 41, 64)        36928     
                                                                 
 max_pooling2d_18 (MaxPoolin  (None, 25, 20, 64)      

In [20]:
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train Model
model.fit(images, labels, batch_size = 1, epochs = 10, validation_split = 0.2)

Epoch 1/10
 269/8000 [>.............................] - ETA: 4:47 - loss: -2572526592.0000 - accuracy: 0.0000e+00

KeyboardInterrupt: 